## 1 

In [18]:
import numpy as np
import random

genome_length = 50000
read_count = 50000
mean_read_length = 250
std_dev_read_length = 30
max_error_rate = 80
num_readings = 100

genome = ''.join(random.choices('ATGC', k=genome_length))

In [19]:
def generate_reads(genome, read_count, mean_length, std_dev_length):
    """генерирует риды"""
    reads = []
    positions = []
    genome_length = len(genome)
    
    for _ in range(read_count):
        read_length = int(np.random.normal(mean_length, std_dev_length))
        read_length = max(1, min(genome_length, read_length))
        
        start_pos = random.randint(0, genome_length - read_length)
        read = genome[start_pos:start_pos + read_length]
        reads.append(read)
        positions.append(start_pos)
    
    return reads, positions

def simulate_sequencing(read, num_readings, max_error_rate):
    """имитирует ошибку при секвенировании"""
    sequenced_read = ""
    correct_reads, error_probability = [], []
    error_count = random.randint(0, max_error_rate)
    
    for i, base in enumerate(read):
        # ошибка в N ридах из 100
        illumina_reads = {"A": 0, "T": 0, "G": 0, "C": 0}
        illumina_reads[base] = 100 - error_count 
        
        for _ in range(error_count):
            if base == "A":
                new_base = random.choice("TGC")
            elif base == "T":
                new_base = random.choice("AGC")
            elif base == "G":
                new_base = random.choice("ATC")
            elif base == "C":
                new_base = random.choice("ATG")
                
            illumina_reads[new_base] += 1
        
        freq_base = max(illumina_reads, key=illumina_reads.get)
        sequenced_read += freq_base
        
        error_probability.append(illumina_reads[freq_base]/100)
        # print(f"read {i} P = {1 - illumina_reads[freq_base]/100}")

        if (freq_base != base):
            correct_reads.append((i, base))

    return sequenced_read, correct_reads, error_probability

In [20]:
true_reads, positions = generate_reads(genome, read_count, mean_read_length, std_dev_read_length)

In [21]:
import math

fastq_data = []
error_data = []

for read_id, read in enumerate(true_reads):
    sequenced_read, mismatched_bases, error_probability = simulate_sequencing(read, num_readings, max_error_rate)
    # https://en.wikipedia.org/wiki/FASTQ_format
    # https://en.wikipedia.org/wiki/Phred_quality_score
    quality_scores = ["~"]*len(read)
    for pos in range(len(read)):
        if error_probability[pos] == 0:
            continue
        quality_scores[pos] = chr(int(-10 * math.log10(error_probability[pos])) + 33)
    
    fastq_entry = f"@read_{read_id}\n{sequenced_read}\n+\n{"".join(quality_scores)}\n"
    fastq_data.append(fastq_entry)

    # сепаратор в файле ошибок - пробел
    error_data_entry = f"Read_ID: {read_id} Start_Pos: {positions[read_id]} Read_Length: {len(read)} Errors: " + ' '.join([f"Pos: {pos} True_Base: {base}" for pos, base in mismatched_bases]) + "\n"
    error_data.append(error_data_entry)

with open("simulated_reads.fastq", "w") as f:
    f.writelines(fastq_data)

with open("error_positions.dat", "w") as f:
    f.writelines(error_data)


Чтобы получить более корректно работающую модель, упростил код

Но, тем не менее, какого-то существенного разброса это не дало. Качество прочтений стабильно низкое

## 2

In [22]:
%%bash
fastqc ./simulated_reads.fastq

Started analysis of simulated_reads.fastq
Approx 5% complete for simulated_reads.fastq
Approx 10% complete for simulated_reads.fastq
Approx 15% complete for simulated_reads.fastq
Approx 20% complete for simulated_reads.fastq
Approx 25% complete for simulated_reads.fastq
Approx 30% complete for simulated_reads.fastq
Approx 35% complete for simulated_reads.fastq
Approx 40% complete for simulated_reads.fastq
Approx 45% complete for simulated_reads.fastq
Approx 50% complete for simulated_reads.fastq
Approx 55% complete for simulated_reads.fastq
Approx 60% complete for simulated_reads.fastq
Approx 65% complete for simulated_reads.fastq
Approx 70% complete for simulated_reads.fastq
Approx 75% complete for simulated_reads.fastq
Approx 80% complete for simulated_reads.fastq
Approx 85% complete for simulated_reads.fastq
Approx 90% complete for simulated_reads.fastq
Approx 95% complete for simulated_reads.fastq
Approx 100% complete for simulated_reads.fastq


Analysis complete for simulated_reads.fastq


File simulated_reads_fastqc.html

          Step options:

               ILLUMINACLIP:<fastaWithAdaptersEtc>:<seed mismatches>:<palindrome clip threshold>:<simple clip threshold>
                   fastaWithAdaptersEtc: specifies the path to a fasta file containing all the adapters, PCR sequences etc.
                   The naming of the various sequences within this file determines how they are used. See below.
                   seedMismatches: specifies the maximum mismatch count which will still allow a full match to be performed
                   palindromeClipThreshold: specifies how accurate the match between the two ´adapter ligated´ reads must be for PE palindrome read alignment.
                   simpleClipThreshold: specifies how accurate the match between any adapter etc. sequence must be against a read.
                   .
                   The adapters are installed on the Debian system at /usr/share/trimmomatic/.

               SLIDINGWINDOW:<windowSize>:<requiredQuality>
                   windowSize: specifies the number of bases to average across
                   requiredQuality: specifies the average quality required.

               LEADING:<quality>
                   quality: Specifies the minimum quality required to keep a base.

               TRAILING:<quality>
                   quality: Specifies the minimum quality required to keep a base.

               CROP:<length>
                   length: The number of bases to keep, from the start of the read.

               HEADCROP:<length>
                   length: The number of bases to remove from the start of the read.

               MINLENGTH:<length>
                   length: Specifies the minimum length of reads to be kept.

           Trimming Order

           Trimming  occurs in the order which the steps are specified on the command line. It is recommended in most cases
           that adapter clipping, if required, is done as early as possible.


In [23]:
%%bash
java -jar ./Trimmomatic-0.39/trimmomatic-0.39.jar SE  ./simulated_reads.fastq trimmomatic.fastq ILLUMINACLIP:TruSeq3-SE:2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36 -phred33 -threads 8

TrimmomaticSE: Started with arguments:
 ./simulated_reads.fastq trimmomatic.fastq ILLUMINACLIP:TruSeq3-SE:2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36 -phred33 -threads 8
java.io.FileNotFoundException: /Data/BioinformaticsCourse2024/homework/2_1/TruSeq3-SE (Нет такого файла или каталога)
	at java.base/java.io.FileInputStream.open0(Native Method)
	at java.base/java.io.FileInputStream.open(FileInputStream.java:216)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:157)
	at org.usadellab.trimmomatic.fasta.FastaParser.parse(FastaParser.java:54)
	at org.usadellab.trimmomatic.trim.IlluminaClippingTrimmer.loadSequences(IlluminaClippingTrimmer.java:110)
	at org.usadellab.trimmomatic.trim.IlluminaClippingTrimmer.makeIlluminaClippingTrimmer(IlluminaClippingTrimmer.java:71)
	at org.usadellab.trimmomatic.trim.TrimmerFactory.makeTrimmer(TrimmerFactory.java:32)
	at org.usadellab.trimmomatic.Trimmomatic.createTrimmers(Trimmomatic.java:59)
	at org.usadellab.trimmomatic.Tri

In [24]:
# Сравнение с учетом того, какой получилась симуляция

In [25]:
from Bio import SeqIO

error_positions = []
with open("error_positions.dat", "r") as f:
    for line in f:
        data = line.split(" ")
        fields = len(data)
        if fields == 7:
            error_positions.append({ "start_pos": int(data[1]), "lenght": int(data[3]), "errors": 0 })
        else:
            ## f"Read_ID: {read_id} Start_Pos: {positions[read_id]} Read_Length: {len(read)} Errors: " + ' '.join([f"Pos: {pos} True_Base: {base}" for pos, base in errors])
            error_positions.append({ "start_pos": int(data[1]), "lenght": int(data[3]), "errors": (fields-7)//2, "error_pos": list(map(int, data[8::4])), "true_base": data[9:4]} )
            # print(data)


def read_fastq(file):
    reads = []
    for record in SeqIO.parse(file, "fastq"):
        reads.append({"id": record.id, "seq": record.seq})
    return reads

def check_match_reads(file1, file2):
    """сравнивает fastq файлы"""
    
    reads1 = read_fastq(file1)
    reads2 = read_fastq(file2)
    all_equal = True
    corrected_count = 0
    
    for read1, read2 in zip(reads1, reads2):
        seq1 = read1["seq"]
        seq2 = read2["seq"]
        
        if (seq1 != seq2):
            all_equal = False
            id = int(read1["id"].split("_", 1)[1])
            true_seq = seq1
            for pos, base in zip(error_positions[id]["error_pos"], error_positions[id]["true_base"]):
                true_seq = true_seq[:pos] + base + true_seq[pos+1:]
            print(f"{read1["id"]} was corrected\tErrors: {sum(1 for a,b in zip(true_seq, seq2) if a != b)}")
            corrected_count += 1
            
    if all_equal:
        print(f"All reads in {file1} and {file2} are match")
    
    return corrected_count

In [26]:
trimm_count = check_match_reads("simulated_reads.fastq", "trimmomatic.fastq")

All reads in simulated_reads.fastq and trimmomatic.fastq are match


In [31]:
print(f"{trimm_count}/50.000 was corrected\nMatch rate is {round(100*trimm_count/50000, 3)}%")

0/50.000 was corrected
Match rate is 0.0%


По сравнению с сгенерированным fastq файлом никаких изменений trimmomatic не внес. Все ошибки остались

## 3
https://www.csd.uwo.ca/~ilie/RACER/

In [33]:
%%bash
./racer/RACER ./simulated_reads.fastq racer.fastq 50000

reading input file...done reading input file
Genome length = 50000
Coverage = 249
Maximum read length = 372
Number of reads = 50000
Number of threads = 8
correcting reads...done correcting reads
writing back the reads...done
Total time building witnesses and counters = 2 seconds.
Total time correcting = 1 seconds.
Total time = 5 seconds.
Peak Memory = 87 MB.


In [36]:
racer_count = check_match_reads("simulated_reads.fastq", "racer.fastq")

read_3 was corrected	Errors: 2
read_21 was corrected	Errors: 8
read_23 was corrected	Errors: 2
read_29 was corrected	Errors: 1
read_43 was corrected	Errors: 20
read_47 was corrected	Errors: 3
read_52 was corrected	Errors: 4
read_75 was corrected	Errors: 5
read_87 was corrected	Errors: 10
read_108 was corrected	Errors: 2
read_200 was corrected	Errors: 5
read_201 was corrected	Errors: 4
read_210 was corrected	Errors: 30
read_218 was corrected	Errors: 21
read_268 was corrected	Errors: 14
read_292 was corrected	Errors: 3
read_302 was corrected	Errors: 1
read_309 was corrected	Errors: 2
read_338 was corrected	Errors: 2
read_357 was corrected	Errors: 5
read_381 was corrected	Errors: 4
read_414 was corrected	Errors: 1
read_418 was corrected	Errors: 1
read_428 was corrected	Errors: 1
read_460 was corrected	Errors: 2
read_482 was corrected	Errors: 3
read_483 was corrected	Errors: 3
read_537 was corrected	Errors: 11
read_544 was corrected	Errors: 5
read_549 was corrected	Errors: 20
read_556 was 

In [37]:
# Проверка на количество измененных ридов
print(f"{racer_count}/50.000 were corrected\nMatch rate is {round(100*racer_count/50000, 3)}%")

2753/50.000 were corrected
Match rate is 5.506%


Данный тул уже вносит изменения

[link](https://neerc.ifmo.ru/wiki/index.php?title=%D0%9E%D1%86%D0%B5%D0%BD%D0%BA%D0%B0_%D0%BA%D0%B0%D1%87%D0%B5%D1%81%D1%82%D0%B2%D0%B0_%D0%B2_%D0%B7%D0%B0%D0%B4%D0%B0%D1%87%D0%B0%D1%85_%D0%BA%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D0%B8#:~:text=FP%20%E2%80%94%20false%20positive%3A%20%D0%BA%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%82%D0%BE%D1%80%20%D0%BD%D0%B5%D0%B2%D0%B5%D1%80%D0%BD%D0%BE,%D0%BD%D0%B5%20%D0%BF%D1%80%D0%B8%D0%BD%D0%B0%D0%B4%D0%BB%D0%B5%D0%B6%D0%B8%D1%82%20%D0%BA%20%D1%80%D0%B0%D1%81%D1%81%D0%BC%D0%B0%D1%82%D1%80%D0%B8%D0%B2%D0%B0%D0%B5%D0%BC%D0%BE%D0%BC%D1%83%20%D0%BA%D0%BB%D0%B0%D1%81%D1%81%D1%83.)


* TP — true positive: классификатор верно отнёс объект к рассматриваемому классу.
* TN — true negative: классификатор верно утверждает, что объект не принадлежит к рассматриваемому классу.
* FP — false positive: классификатор неверно отнёс объект к рассматриваемому классу.
* FN — false negative: классификатор неверно утверждает, что объект не принадлежит к рассматриваемому классу.

рассматриваемый класс - нуклеотид в правильном риде
* TP - после работы тула нуклеотид такой же как в оригинальном риде
* TN - иллюмина была с ошибкой. Тул увидел, попробовал исправить, но неудачно

* FP - 
* FN - ???



Не могу сформулировать, что в данном случае понимать за fp, fn, поэтому буду отталкиваться от следующей матрицы ошибок

| Геном (факт) \ Предсказание тула | Предсказано изменение относительно illumina | Не предсказано |
| --- | --- | --- |
| Совпадает | TP | FP |
| НЕсовпадает | FN | TN |

In [38]:
def check_rate(file2, true_reads=true_reads):
    """сравнивает fastq файл с корректными ридами и считает метрики TP, TN, FP, FN"""
    
    file1 = "simulated_reads.fastq"
    reads1 = read_fastq(file1)
    reads2 = read_fastq(file2)
    
    TP, TN, FP, FN = 0, 0, 0, 0

    count = 0
    for true_seq, read1, read2 in zip(true_reads, reads1, reads2):
        illumina_seq = read1["seq"]
        corrected_seq = read2["seq"]
        for t, i, c in zip(true_seq, illumina_seq, corrected_seq):
            count += 1

            if c != i:
                if t == c:
                    TP += 1
                else:
                    FN += 1   
            elif c == i:
                if c == t:
                    TP += 1
                else:
                    FN += 1
                

    assert count == TP + TN + FP + FN
    procent = lambda x: str(round(x, 3)) + "%"

    print(f"TP = {TP}\t or {procent(100*TP/count)}")
    print(f"TN = {TN}\t or {procent(100*TN/count)}")
    print(f"FP = {FP}\t or {procent(100*FP/count)}")
    print(f"FN = {FN}\t or {procent(100*FN/count)}")
   

In [39]:
# метрики по 3 заданию
# сравнение по всем нуклеотидам
check_rate("racer.fastq")

TP = 11183255	 or 89.638%
TN = 0	 or 0.0%
FP = 0	 or 0.0%
FN = 1292822	 or 10.362%


## 4 

Racer сделал множество сомнительных исправлений, по отработал быстрее


На графиках в исходной статье он находится отдельно от остальных инструментов, работая существенно быстрее при меньшей точности - здесь, в целом, поведение такое же как в статье